In [ ]:
import torch
import re

from transformers import AutoTokenizer, AutoProcessor, AutoModelForCausalLM

In [ ]:
def check_lora_weight_diff(baseline_model, finetuned_model, atol=1e-6):
    modules_to_check = [
        "model.multi_modal_projector.linear_1",
        "model.multi_modal_projector.linear_2",
    ]

    baseline_params = dict(baseline_model.named_parameters())
    finetuned_params = dict(finetuned_model.named_parameters())

    for module_name in modules_to_check:
        if f"{module_name}.weight" not in baseline_params:
            print(f"⚠️ Skipping {module_name}, not found in baseline model")
            continue
        if f"{module_name}.weight" not in finetuned_params:
            print(f"⚠️ Skipping {module_name}, not found in finetuned model")
            continue

        baseline_weight = baseline_params[f"{module_name}.weight"].detach().cpu().float()
        finetuned_weight = finetuned_params[f"{module_name}.weight"].detach().cpu().float()

        diff = (finetuned_weight - baseline_weight).abs()
        max_diff = diff.max().item()
        mean_diff = diff.mean().item()

        print(f"🔍 Checking {module_name}:")
        print(f"   Max abs diff:  {max_diff:.6e}")
        print(f"   Mean abs diff: {mean_diff:.6e}")
        if torch.allclose(baseline_weight, finetuned_weight, atol=atol):
            print("   ✅ No significant changes (within tolerance)\n")
        else:
            print("   ⚠️ Weights changed (LoRA seems applied)\n")




def manual_inference(model, processor, image, prompt):


    inputs = processor(prompt, image, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=False,

        )


    generated_text = processor.decode(output[0], skip_special_tokens=True)
    return generated_text